# Notebook de pré-processamento da planilha IW47

### Importando bibliotecas necessárias

In [ ]:
import pandas as pd
from unidecode import unidecode

### Leitura da planilha

In [ ]:
path = "../../../data/"
pathIn = path + "input/"
pathUtil = path + "util/"

df = pd.read_excel(
    pathIn + "IW47-Confirmações_Executadas_2020-a-28jun24.xlsx", sheet_name="Sheet1", engine="calamine"
)

### Pré-processamento da coluna 'Txt.breve operação'

In [ ]:
print(df["Txt.breve operação"].value_counts())

In [ ]:
# Converte coluna 'Txt.breve operação' para lowercase e remove acentos
df["Txt.breve operação"] = df["Txt.breve operação"].apply(
    lambda x: unidecode(x).lower() if isinstance(x, str) else x)


# Refatora coluna 'Txt.breve operação'
# Por exemplo:
#    Txt.breve operação    Output
# 1: "servico-6a"       => "servico"
df["Txt.breve operação"] = df["Txt.breve operação"].str.split("-").str[0]
df["Txt.breve operação"] = df["Txt.breve operação"].str.split(" ").str[0]

#### Transforma linhas com o prefixo das operações mais comuns para o seu nome padrão

In [ ]:
dict_prefixos = {
    "serv": "servico",
    "desl": "deslocamento",
    "prep": "preparo",
    "rel": "relatorio",
    "super": "supervisao",
    "lab": "laboratorio",
    "eng": "engenharia",
    "exec": "execucao",
    "comiss": "comissionamento",
    "insp": "inspecao",
    "verif": "verificacao",
    "planej": "planejamento",
    "calib": "calibracao"
}

# Atualiza os valores da coluna 'Txt.breve operação' de acordo com o dicionário 'dict_prefixos'


def atualizar_coluna(valor, dict_prefixos):
    for prefixo, novo_valor in dict_prefixos.items():
        if str(valor).startswith(prefixo):
            return novo_valor
    # Modifica os valores de 'Txt.breve operação' menos frequentes para 'outros'
    return "outros"


df["Txt.breve operação"] = df["Txt.breve operação"].apply(
    lambda x: atualizar_coluna(x, dict_prefixos))

#### Cria um dataset para validar a frequencia de ocorrências das principais operações

In [ ]:
# Cria dataframe com a contagem de valores únicos da coluna 'Txt.breve operação'
df_conta_valores_unicos = df["Txt.breve operação"].value_counts().reset_index()

df_conta_valores_unicos.columns = ["Txt.breve operação", "count"]

total = df_conta_valores_unicos["count"].sum()

# Cria coluna 'percentual' com a porcentagem de ocorrências de cada valor
df_conta_valores_unicos["percentual"] = round(
    (df_conta_valores_unicos["count"] / total) * 100, 2)

# soma os 11 primeiros valores
df_conta_valores_unicos.head(11)["percentual"].sum()

### Salva o DataFrame da IW47 processado como csv

In [ ]:
# Salva o dataframe em um arquivo .csv
df.to_csv(pathUtil + "os/IW47_Executadas_processado.csv", index=False)